In [89]:
import os
import django
from django.conf import settings

# Set the Django settings module
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "sv_carbon_removal.settings")
django.setup()


In [90]:
from django.conf import settings

In [91]:
settings

<LazySettings "sv_carbon_removal.settings">

In [217]:
settings.MEDIA_URL

'/media/'

In [218]:
from asgiref.sync import sync_to_async
from fcd_gee.models import ProjectCarbon, SatVerConfiguration

async def my_async_function():
    # Async function code here

    # Convert the synchronous queryset to asynchronous using sync_to_async
    queryset = await sync_to_async(list)(SatVerConfiguration.objects.all())

    # Process the queryset asynchronously
    for obj in queryset:
        # Async processing code here
        # print(obj.id, obj.start_date_analysis)
        shp_aoi = await sync_to_async(lambda: obj.shp_aoi)()
        print(obj.id, shp_aoi.id, shp_aoi.file)
    return queryset

# Call the async function
query = await my_async_function()

58 39 uploads/aoi_TEST_DEMO_TREEO_shp_axis_admin.zip


In [219]:
query

[<SatVerConfiguration: TEST_DEMO_TREEO_Landsat_2022>]

In [220]:
conf_selected = [sel for sel in query if sel.label == 'TEST_DEMO_TREEO_Landsat_2022'][0]

In [221]:
conf_selected

<SatVerConfiguration: TEST_DEMO_TREEO_Landsat_2022>

In [222]:
type(conf_selected.start_date_analysis)

datetime.date

In [223]:
import ee
import os
os.getcwd()

'/home/mfirdaus/gee_mapbox/backend/sv_carbon_removal'

In [224]:
# Trigger the authentication flow. if you want to user json, please comment this
# ee.Authenticate()
# Initialize the library.
import os

service_account = 'iqbalpythonapi@bukit30project.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, os.path.join(os.getcwd(),'fcd_gee/bukit30project-4d92e5b46ea7.json'))

# Initialize the library.
# ee.Initialize(project='bukit30project')
ee.Initialize(credentials)

# ee.Initialize(project='bukit30project')

In [225]:
import math
from datetime import datetime

In [226]:
# class cloudmasking for landsat
def cloud_mask_tm(image, sr=True):
    qa = image.select('QA_PIXEL')
    dilated = 1 << 1
    cloud = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(dilated).eq(0) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))

    if sr:
        optical_bands = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']) \
                                            .updateMask(mask) \
                                            .multiply(0.0000275).add(-0.2)


        thermal_band = image.select(['ST_B6'], ['thermal']).updateMask(mask) \
                                            .multiply(0.00341802).add(149.0)


    else:
        optical_bands = image.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']) \
                                            .updateMask(mask) \
                                            .multiply(0.0001)

        thermal_band = image.select(['B6'], ['thermal']).updateMask(mask) \
                                            .multiply(0.0001)

    cloudM = mask.rename('cloudM')

    imageCloudMasked = image.addBands(optical_bands, None, True) \
                            .addBands(thermal_band, None, True)

    imageCloudMasked = imageCloudMasked.addBands(cloudM)

    return imageCloudMasked

def cloud_mask_oli(image, sr=True):
    qa = image.select('QA_PIXEL')
    dilated = 1 << 1
    cirrus = 1 << 2
    cloud = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(dilated).eq(0) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))

    if sr:
        #print('sr=True')
        optical_bands = image.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']) \
                        .updateMask(mask) \
                        .multiply(0.0000275).add(-0.2)

        thermal_band = image.select(['ST_B10'], ['thermal']).updateMask(mask) \
                        .multiply(0.00341802).add(149.0)

    else:
        optical_bands = image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'], ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']) \
                        .updateMask(mask) #\
                        #.multiply(0.0001)

        # image = detect_cloud_shadow(image)

        thermal_band = image.select(['B10'], ['thermal']) \
                        .updateMask(mask) #\
                        #.multiply(0.0001)

        #best_image_year = detect_cloud_shadow(best_image_year)

        # thermal index adapted from:

        AlB10 = ee.Number(image.get('RADIANCE_ADD_BAND_10'))
        M1B10 = ee.Number(image.get('RADIANCE_MULT_BAND_10'))

        radiance = thermal_band.multiply(M1B10).add(AlB10)

        K_1 = ee.Number(image.get('K1_CONSTANT_BAND_10'))
        K_2 = ee.Number(image.get('K2_CONSTANT_BAND_10'))
        A = ee.Number(image.get('RADIANCE_ADD_BAND_10'))
        M = ee.Number(image.get('RADIANCE_MULT_BAND_10'))

        # Getting the Thermal Index - source??
        TI = radiance.expression('(K_2) / log((K_1 / TEMP)+1)',{
              'K_2': K_2,
              'K_1': K_1,
              'TEMP':radiance
            }
          ).rename('TI')


    cloudM = mask.rename('cloudM')

    imageCloudMasked = image.addBands(optical_bands, None, True) \
                            .addBands(thermal_band, None, True)
    if sr==False:
        #print('sr=False')
        imageCloudMasked = imageCloudMasked.addBands(TI, None, True)

    imageCloudMasked = imageCloudMasked.addBands(cloudM)

    return imageCloudMasked

In [227]:
# cloud masked func. Sentinel

#CLOUD_FILTER = 80
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def apply_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    not_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 30})
        .Not().rename('cloudM'))

    # Add the final cloud-shadow mask to the image.
    #img_all = img_cloud_shadow.addBands(not_cld_shdw)

    # Preserve specific properties by copying from the original image
    preserved_properties = img.propertyNames()
    img_masked = img_cloud_shadow.select(['B.*']).updateMask(not_cld_shdw).multiply(0.0001)

    # Copy the preserved properties back to the masked image
    img_masked = img_masked.copyProperties(img, preserved_properties)

    return {'masked_image': img_masked, 'cloudM': not_cld_shdw}

In [228]:
# class imagecollection composite
class ImageCollectionComposite:
    def __init__(self, AOI=None, date_start_end=['2022-1-1',"2022-12-31"], cloud_cover_threshold = 40):
        self.AOI = AOI

        # get the date based on the input list
        self.date_start_end = date_start_end

        import datetime 

        self.start_date = self.date_start_end[0]
        if isinstance(self.start_date, datetime.date):
            self.start_date = self.start_date.strftime("%Y-%m-%d")           

        self.end_date = self.date_start_end[1]
        if isinstance(self.end_date, datetime.date):
            self.end_date = self.end_date.strftime("%Y-%m-%d")      

        self.date_object = datetime.datetime.strptime(self.end_date, "%Y-%m-%d")
        self.year = self.date_object.year

        self.cloud_cover_threshold = cloud_cover_threshold

    # utils for filter collection
    def filter_collection(self, col):
        return col.filterDate(self.start_date, self.end_date).filterBounds(self.AOI). \
                                                              filter(ee.Filter.lte('CLOUD_COVER', self.cloud_cover_threshold))

    def merging_collection_landsat(self):
        # col -> or collection
        l4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
        l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
        l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
        l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")

        l4_raw = ee.ImageCollection("LANDSAT/LT04/C02/T1")
        l7_raw = ee.ImageCollection("LANDSAT/LE07/C02/T1")
        l9_raw = ee.ImageCollection("LANDSAT/LC09/C02/T1")
        l8_raw = ee.ImageCollection("LANDSAT/LC08/C02/T1")
        l5_raw = ee.ImageCollection("LANDSAT/LT05/C02/T1")

        # Composite function and only use cloud_masked image
        def landsat457():
            collection = self.filter_collection(l4).merge(self.filter_collection(l5)).merge(self.filter_collection(l7))
            image_collection = collection.map(cloud_mask_tm)#.median().clip(roi)
            return image_collection

        def landsat89():
            collection = self.filter_collection(l8).merge(self.filter_collection(l9))
            image_collection = collection.map(cloud_mask_oli)#.median().clip(roi)
            return image_collection

        def landsat_raw457():
            collection = self.filter_collection(l4_raw).merge(self.filter_collection(l5_raw)).merge(self.filter_collection(l7_raw))
            image_collection = collection.map(lambda image: cloud_mask_tm(image, sr=False))#.median().clip(roi)
            return image_collection

        def landsat_raw89():
            collection = self.filter_collection(l8_raw).merge(self.filter_collection(l9_raw))
            image_collection = collection.map(lambda image: cloud_mask_oli(image, sr=False))#.median().clip(roi)
            return image_collection

        print('selecting LANDSAT images')
        # select based on year, landsat 8 and 9 will be merged in the year 2014 or more
        if self.year < 2014:
            landsat = landsat457
            landsat_raw = landsat_raw457
        else:
            landsat = landsat89
            landsat_raw = landsat_raw89

        image_col = landsat() # this one without median aggregation, still need to do it after,
        return image_col

    # planet labs use the existing basemap, without masking, later will be treated in median
    def merging_collection_planet(self, region='asia'):
        print('selecting Planet images')
        nicfi_col = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia")
        if region == 'asia':
            nifci_col = nicfi_col
        elif region == 'africa':
            nicfi_col = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa")

        filteredNicfi = nicfi_col.filterDate(self.start_date, self.end_date) \
                     .filterBounds(self.AOI)

        def scaling_planet(image):
            sr_image = image.select(['R','G','B','N'], ['red','green', 'blue','nir']).multiply(0.0001)
            return sr_image

        filtered_scaled = filteredNicfi.map(scaling_planet) # this one already in median, single image
        return filtered_scaled

    def merging_collection_sentinel(self):
        print('selecting Sentinel images')

        s2_sr_cld_col = get_s2_sr_cld_col(self.AOI,self.start_date, self.end_date)

        SEN_BANDS = ['B2',   'B3', 'B4',  'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'cloudM' ]
        bandNamesSentinel2 = ['blue', 'green', 'red', 'redE1', 'redE2', 'redE3', 'nir', 'redE4', 'swir1', 'swir2','cloudM']

        # Apply the cloud and shadow mask to the entire image collection
        s2_sr_cld_col_masked = s2_sr_cld_col.map(lambda image: apply_cld_shdw_mask(image)['masked_image'])
        cloudM = s2_sr_cld_col.map(lambda image: apply_cld_shdw_mask(image)['cloudM'])

        # Convert the image collections to lists
        s2_sr_cld_list = s2_sr_cld_col_masked.toList(s2_sr_cld_col_masked.size())
        cloudM_list = cloudM.toList(cloudM.size())

        # Create an empty list to store the merged images
        merged_images = []

        # Iterate through the images in the collections and merge them
        for i in range(s2_sr_cld_col_masked.size().getInfo()):
            masked_image = ee.Image(s2_sr_cld_list.get(i))
            cloud_mask = ee.Image(cloudM_list.get(i))
            merged_image = masked_image.addBands(cloud_mask.rename('cloudM'))
            merged_images.append(merged_image)

        # Create a new ImageCollection from the merged images list
        s2_sr_cld_col_merged = ee.ImageCollection(merged_images)

        # Select the desired bands from the first image
        s2_sr_col = s2_sr_cld_col_merged.select(SEN_BANDS, bandNamesSentinel2)

        return s2_sr_col

In [229]:
# topographic correction for landsat
def radians(img):
    return img.toFloat().multiply(3.14159265359).divide(180)

# Define the function to perform physical correction
def doPhysicalCorrection(image, type_DEM='srtm', I_satellite='Landsat'):

    # illumination parameters
    alos = ee.Image('JAXA/ALOS/AW3D30_V1_1')
    srtm = ee.Image('USGS/SRTMGL1_003')

    extent = image.geometry().bounds(1)

    # Select DEM, calculate aspect and slope

    if type_DEM == 'alos':
        DEM = alos
    else:
        DEM = srtm

    DEM = DEM.clip(extent)
    boxcar = ee.Kernel.square(radius=3, units='pixels', normalize=True)
    DEMs = DEM.convolve(boxcar)
    SLP_deg = ee.Terrain.slope(DEMs)
    SLP = radians(SLP_deg)
    ASP_deg = ee.Terrain.aspect(DEMs)
    ASP = radians(ASP_deg)
    cos_SLP = SLP.cos()

    if I_satellite == 'Landsat':
        AZ = ee.Number(image.get('SUN_AZIMUTH'))
        ZE = ee.Number(image.get('SUN_ELEVATION'))
    elif I_satellite == 'Sentinel':
        AZ = ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE'))
        ZE = ee.Number(image.get('MEAN_SOLAR_ZENITH_ANGLE'))

    AZ_R = radians(ee.Image(AZ)).clip(extent)
    ZE_R = radians(ee.Image(ZE)).clip(extent)
    cos_ZE = ZE_R.cos()
    cos_ZE_SLP = cos_ZE.multiply(SLP.cos())
    cos_VA = ee.Image(0).clip(extent).cos()

    # Calculate cos(Z)⋅cos(s) + sin(Z)⋅sin(s)⋅cos(a - a')
    IL1 = AZ_R.subtract(ASP).cos().multiply(SLP.sin()).multiply(ZE_R.sin()).add(cos_ZE.multiply(SLP.cos()))
    IL2 = IL1.where(IL1.lte(0), 0)
    IL3 = IL2.where(IL2.gt(0), 1)
    IL = IL1.mask(IL3).select([0], ['IL'])

    # Create and return a dictionary
    keyList = ee.List(['IL', 'cos_ZE', 'cos_VA', 'cos_SLP', 'SLP', 'ASP'])
    valueList = ee.List([IL, cos_ZE, cos_VA, cos_SLP, SLP, ASP])
    dictionary = ee.Dictionary.fromLists(keyList, valueList)

    # get illumination parameter result
    parameters = dictionary

    # starting to apply the parameter
    cos_ZE = ee.Image(parameters.get('cos_ZE'))
    cos_VA = ee.Image(parameters.get('cos_VA'))
    cos_SLP = ee.Image(parameters.get('cos_SLP'))

    IL = ee.Image(parameters.get('IL'))

    imgTC = image.select().addBands(image.select(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])  \
                                        .mask(image.select(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])  \
                                        .And(IL).And(image.select('cloudM')))    \
                                        .multiply((((cos_ZE).add(cos_VA))).divide((IL.add(cos_SLP)))))   \
                                        .addBands(image.select('cloudM'))   \
                                        .set('system:time_start', image.get('system:time_start'))

    return ee.Image(imgTC)

In [230]:
class SpectralAnalysis:
    def __init__(self, image, AOI = None, I_satellite= 'Landsat', pca_scaling = 1, tileScale = 1):
        self.image = image
        self.I_satellite = I_satellite
        self.AOI = AOI

        # scaling the pixel per satellite vendor type
        self.pca_scaling = pca_scaling
        self.pca_scale = 1*30
        if self.I_satellite == 'Landsat':
            self.pca_scale = self.pca_scaling *30 # 1 meaning that 1 x pixel size of spatial resolution e.g., Planet Labs 1 x 5, Sentinel 1 x 10, Landsat 1 x 30
        elif self.I_satellite == 'Sentinel':
            self.pca_scale = self.pca_scaling *10
        elif self.I_satellite == 'Planet':
            self.pca_scale = self.pca_scaling *5

        self.tileScale = tileScale # increase this if user memory limit occur, see: https://gis.stackexchange.com/questions/373250/understanding-tilescale-in-earth-engine

    def max_bands(self):
        maxRed = self.image.select('red').reduceRegion(reducer=ee.Reducer.max(),
                                                        geometry=self.AOI,
                                                        scale=self.pca_scale,
                                                        bestEffort=True,
                                                        tileScale=self.tileScale)
        #print("MAXRED",maxRed)
        maxRed = ee.Dictionary(maxRed).toImage()
        maxGreen =self.image.select('green').reduceRegion(reducer=ee.Reducer.max(),
                                                    geometry=self.AOI,
                                                    scale=self.pca_scale,
                                                    bestEffort=True,
                                                    tileScale=self.tileScale)
        maxGreen = ee.Dictionary(maxGreen).toImage()
        maxBlue = self.image.select('blue').reduceRegion(reducer=ee.Reducer.max(),
                                                    geometry=self.AOI,
                                                    scale=self.pca_scale,
                                                    bestEffort=True,
                                                    tileScale=self.tileScale)
        maxBlue = ee.Dictionary(maxBlue).toImage()
        return [maxRed,maxGreen,maxBlue]


    # Advanced Vegetation index - scaled from old formula (8bit) to relative number based on region
    def AVI_func(self):
        AVI = self.image.expression('((nir + 1 )* (maxRed-red) *(nir -red))**0.333',{
            'nir':self.image.select(['nir']),
            'red':self.image.select(['red']),
            'maxRed': self.max_bands()[0],
            }
        ).rename('AVI')

        return AVI

    # Bare Soil Index
    # https://www.researchgate.net/publication/319045433_Integration_of_GIS_and_Remote_Sensing_for_Evaluating_Forest_Canopy_Density_Index_in_Thai_Nguyen_Province_Vietnam/link/599293be0f7e9b433f415b40/download
    def BSI_func(self):
        BSI = ''
        if self.I_satellite == 'Landsat' or self.I_satellite == 'Sentinel':
            BSI = self.image.expression('(((SWIR1 + red) - (G + NIR)) / ((SWIR1 + red) + (G + NIR)))*100+100',{
                'SWIR1':self.image.select(['swir1']),
                'G':self.image.select(['green']),
                'red':self.image.select(['red']),
                'NIR':self.image.select(['nir']),
                }
            ).rename('BSI')

        elif self.I_satellite == 'Planet':
            BSI = self.image.expression(
                '(NIR + GREEN + RED) / (NIR + GREEN - RED)',  {
                    'NIR': self.image.select('nir'),
                    'GREEN': self.image.select('green'),
                    'RED': self.image.select('red')
                        }
                    ).rename('BSI')

        return BSI

    def SI_func(self):
        SI = self.image.expression('((maxGreen-Green)*(maxBlue-Blue)*(maxRed-Red))**(1/3)',{
            'Red':self.image.select(['red']),
            'Green':self.image.select(['green']), #//******
            'Blue':self.image.select(['blue']),
            'maxRed': self.max_bands()[0],
            'maxGreen': self.max_bands()[1],
            'maxBlue': self.max_bands()[2],
        }
        ).rename('SI')
        return SI

In [231]:
# normalization image, utils
# adapted from https://code.earthengine.google.com/a6013a970da3901f42b1db8ae7fc265a
def normalization_100(image, pca_scale=30, AOI=None):
    #image_scale = ee.Number(image.projection().nominalScale())
    image_scale = pca_scale
    region = AOI
    def normalize_band(name):
        name = ee.String(name)
        band = image.select(name)

        mean_std = image.reduceRegion(
            reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            geometry=region,
            scale=image_scale,
            maxPixels=10e9
        )

        mean = ee.Number(mean_std.get(name.cat('_mean')))
        std = ee.Number(mean_std.get(name.cat('_stdDev')))

        max_value = mean.add(std.multiply(3))
        min_value = mean.subtract(std.multiply(3))

        band1 = ee.Image(min_value).multiply(band.lt(min_value)).add(ee.Image(max_value).multiply(band.gt(max_value))) \
            .add(band.multiply(ee.Image(1).subtract(band.lt(min_value)).subtract(band.gt(max_value))))

        result_band = band1.subtract(min_value).divide(max_value.subtract(min_value))

        return result_band

    band_names = image.bandNames()
    unit_scale = ee.ImageCollection.fromImages(band_names.map(normalize_band)).toBands().rename(band_names)
    return unit_scale.multiply(100)

def assigning_band(band_name_image,class_value,srcImg):

    # Create an image with the constant value for class
    constant_image_class = srcImg.multiply(0).add(class_value).rename(band_name_image)
    constant_image_pixel = srcImg.multiply(0).add(1).rename('pixel')

    # Add the new band to the existing image
    pixel_bandimg= srcImg.addBands(constant_image_pixel)
    pix_classImg = pixel_bandimg.addBands(constant_image_class)
    pix_classImg = pix_classImg.select([band_name_image,'pixel'])

    return pix_classImg

In [232]:
file_location_conf = await sync_to_async(lambda: conf_selected.shp_aoi.file)()

In [233]:
file_location_conf.path

'/home/mfirdaus/gee_mapbox/backend/sv_carbon_removal/media/uploads/aoi_TEST_DEMO_TREEO_shp_axis_admin.zip'

In [234]:
type(file_location_conf)

django.db.models.fields.files.FieldFile

In [235]:
print(file_location_conf)

uploads/aoi_TEST_DEMO_TREEO_shp_axis_admin.zip


In [236]:
import zipfile
import tempfile

def unzip_file(zip_file):
    """Extracts files from a zip archive without the top-level parent folder.

    Args:
        zip_file (str): The path to the zip file.
        destination (str): The path to the destination directory.
    """

    temp_dir = tempfile.mkdtemp()

    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            # Skip directories to avoid extracting the parent folder
            if file_info.is_dir():
                continue

            # Extract the file, removing the top-level parent folder from the path
            file_path = file_info.filename
            base_filename = os.path.basename(file_path)  # Get the base filename
            extracted_path = os.path.join(temp_dir, base_filename)  # Construct full path without parent folder

            # Create any necessary parent directories
            os.makedirs(os.path.dirname(extracted_path), exist_ok=True)

            with open(extracted_path, 'wb') as outfile:
                outfile.write(zip_ref.read(file_info))

    return temp_dir

In [237]:
temp_dir = unzip_file(file_location_conf.path)

In [186]:
# !cd /tmp/tmpppxot7os && ls

axis.dbf  axis.prj  axis.shp  axis.shx


In [238]:
os.listdir(temp_dir)

['axis.dbf', 'axis.shp', 'axis.shx', 'axis.prj']

In [239]:
# TEST VARIABLE TO GEE FROM DJANGO
start_date = conf_selected.start_date_analysis
end_date = conf_selected.end_date_analysis

# for planet labs image
region = 'africa' # or you can use 'asia'

# cloud cover threshold, max per image, in image collection
cloud_cover_threshold = 40

import geemap
# input shp region based on the AOI, relative path to our collab folder content
# AOIt_shp = '/content/drive/MyDrive/shp_local/AOI_AXIS_BIG.shp'

list_files_tmp = os.listdir(temp_dir)
shp_tmp_text = [f for f in list_files_tmp if '.shp' in f][0]
AOIt_shp = os.path.join(temp_dir, shp_tmp_text)

AOIt_shp_plot = geemap.shp_to_ee(AOIt_shp)
crs_input = 'EPSG:4326'

AOI = AOIt_shp_plot


In [240]:
# choose the satellite image you want to use
I_satellite = conf_selected.satellite_use

In [241]:
I_satellite

'Landsat'

In [242]:
#NDWI water limit
ndwi_hi_sentinel = 0.05 # for Sentinel
ndwi_hi_landsat = 0.1 # for landsat
ndwi_hi_planet = -0.2 # for Planet Labs

ndwi_hi = 0.1
if I_satellite == 'Landsat':
    ndwi_hi = ndwi_hi_landsat
elif I_satellite == 'Sentinel':
    ndwi_hi = ndwi_hi_sentinel
elif I_satellite == 'Planet':
    ndwi_hi = ndwi_hi_planet

pca_scaling = 1  # 1 meaning that 1 x pixel size of spatial resolution e.g., Planet Labs 1 x 5, Sentinel 1 x 10, Landsat 1 x 30
tileScale = 1 # increase this if user memory limit occur, see: https://gis.stackexchange.com/questions/373250/understanding-tilescale-in-earth-engine

######## HANSEN Tree Cover and Tree Cover Loss Input - Historical Data Check (10 Years Rule) ###################################################
# Hansen 10 years rule and Forest - Tree Cover Hansen
# define tree cover minimum that classified as forest e.g., Indonesia > 30%, hence 30
tree_cover_forest = 30 # this will be in percent, let's say, forest is > 30% if you put 30
pixel_number = 3 #define minimum mapping unit classified as forest, 1 pixel for landsat 30mx30, 3 pixel is forest ~> 0.27 Ha
year_start_loss = 13 #define start year to track as 10 years rule (e.g., 2012 to 2022 (track 10 years rule), hence 12. format= 00->2000, 12->2012)


## FCD Threshold
high_forest = 65
yrf_forest = 45
shrub_grass = 25

# output Band Names
band_name_image = 'Class'

#for area id in shapefile that identified the data, and will converted into raster
OID = 'id'  #IMPORTANT TO CHECK OID based on the column ID
#############################################
##################################################################################
### Masking and overlay and area helper Make an image out of the AOI area attribute -> convert featurecollection into raster (image) for overlaying tools
AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
    properties= [OID],
    reducer= ee.Reducer.first()
)

In [243]:
# initiate instance class for the image collection and later mosaicking
classInputCollection = ImageCollectionComposite(AOI=AOI, date_start_end=[start_date,end_date], cloud_cover_threshold = cloud_cover_threshold)

In [244]:
# performing the merge_collection method based on the I_satellite
image_collection_mask = None
if I_satellite == 'Landsat':
    image_collection_mask = classInputCollection.merging_collection_landsat()
elif I_satellite == 'Planet':
    image_collection_mask = classInputCollection.merging_collection_planet(region) # if there is no image in here, maybe you choose a WRONG REGION
elif I_satellite == 'Sentinel':
    image_collection_mask = classInputCollection.merging_collection_sentinel()

selecting LANDSAT images


In [255]:
a = {'test':'ssss', 'test2':'sssssa223'}
a.test

AttributeError: 'dict' object has no attribute 'test'

In [245]:
image_mosaick = image_collection_mask.median().clip(AOI)
# we will use landsat that already corrected its topography
if I_satellite == 'Landsat':
    image_mosaick = image_collection_mask.map(lambda image: doPhysicalCorrection(image, type_DEM='srtm',
                                                                                  I_satellite='Landsat')) \
                                                                                  .median().clip(AOI)

# initialize Map object
Map = geemap.Map(center=(-3, 115), zoom=4)
Map.centerObject(AOI, 10)

In [246]:
# add to canvas Map for this mosaic layer
if I_satellite == 'Planet':
    # true color {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}
    # nir veg color {"bands":["red","nir","blue"],"min":0,"max":0.6,"gamma":1.5 }
    Map.addLayer(image_mosaick,{"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}, f'{I_satellite} mosaicked - {start_date}-{end_date} VegColor')
else:
    Map.addLayer(image_mosaick,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5 }, f'{I_satellite} mosaicked - {start_date}-{end_date}')

In [247]:
Map

Map(center=[0.288790675212759, 31.699612779925186], controls=(WidgetControl(options=['position', 'transparent_…

In [248]:
classImageSpectral = SpectralAnalysis(image_mosaick, AOI = AOI, I_satellite= I_satellite, pca_scaling = pca_scaling, tileScale = tileScale)

In [249]:
#############################
## Forest Canopy Density Mapping
#############################
#### PRINCIPAL COMPONENT ANALYSIS ########
# adapted from https://developers.google.com/earth-engine/guides/arrays_eigen_analysis
# Define the mean-centered function
def means_centered(image, region, scale, tileScale):
    bandNames = image.bandNames()

    # Mean center the data
    meanDict = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=scale,
        maxPixels=1e9,
        tileScale=tileScale,
        bestEffort=True,
    )
    means = ee.Image.constant(meanDict.values(bandNames))
    centered = image.subtract(means)

    return [centered, bandNames]

# Define the helper function to generate new band names
def getNewBandNames(prefix, bandNames):
    seq = ee.List.sequence(1, bandNames.length())
    return seq.map(lambda b: ee.String(prefix).cat(ee.Number(b).int().format()))

# Define the function to get principal components
def getPrincipalComponents(centered, scale, region, bandNames):
    # Collapse the bands into a 1D array per pixel
    arrays = centered.toArray()

    # Compute the covariance within the region
    covar = arrays.reduceRegion(
        reducer=ee.Reducer.centeredCovariance(),
        geometry=region,
        scale=scale,
        maxPixels=1e9,
        tileScale=tileScale,
        bestEffort=True,

    )

    # Get the covariance array result and cast to an array
    covarArray = ee.Array(covar.get('array'))

    # Perform eigen analysis and slice apart the values and vectors
    eigens = covarArray.eigen()

    # Extract eigenvalues and eigenvectors
    eigenValues = eigens.slice(1, 0, 1)
    eigenVectors = eigens.slice(1, 1)

    # Convert the array image to 2D arrays
    arrayImage = arrays.toArray(1)

    # Left multiply the image array by the matrix of eigenvectors
    principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

    # Square root of eigenvalues as a P-band image
    sdImage = ee.Image(eigenValues.sqrt()) \
        .arrayProject([0]).arrayFlatten([getNewBandNames('sd', bandNames)])

    # Normalize the principal components by their standard deviations
    return principalComponents \
        .arrayProject([0]) \
        .arrayFlatten([getNewBandNames('pc', bandNames)]) \
        .divide(sdImage)

print('processing AVI')
avi_image = classImageSpectral.AVI_func()
#Map.addLayer(avi_image,{},'avi')

print('processing BSI')
bsi_image = classImageSpectral.BSI_func()
#Map.addLayer(bsi_image,{},'bsi')

print('processing SI')
si_image = classImageSpectral.SI_func()
#Map.addLayer(si_image,{},'si_no_norm')

#Map.addLayer(image_corr_8,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 256,  }, f'landsat_for_FCD_{year}_corrected_8bit')
print('Normalizing to 100 AVI')
##### Starting (normalized 100) Indices
# acquire the existing configuration after conditional I_satellite
pca_scale = classImageSpectral.pca_scale
tileScale = classImageSpectral.tileScale


avi_norm = normalization_100(avi_image, pca_scale=pca_scale, AOI=AOI)
Map.addLayer(avi_norm,{'min':0,'max':100},'avi_norm', False)
print('Normalizing to 100 BSI')
bsi_norm = normalization_100(bsi_image, pca_scale=pca_scale, AOI=AOI)
Map.addLayer(bsi_norm,{'min':0,'max':100},'bsi_norm', False)
print('Normalizing to 100 SI')
si_norm = normalization_100(si_image, pca_scale=pca_scale, AOI=AOI)
Map.addLayer(si_norm, {'min':0,'max':100},'si_norm', False)
print('Combining AVI AND BSI')
# Combine  AVI and BSI to one image with two bands
AVI_BSI = avi_norm.addBands(bsi_norm)
# Masked-out process or remove null data, to avoid errors
avi_bsi_clean = AVI_BSI.gte(0).Or(AVI_BSI.lte(0))
AVI_BSI = AVI_BSI.updateMask(avi_bsi_clean)
#Map.addLayer(AVI_BSI, {'min':0,'max':100}, 'AVI_BSI')

print('Processing means center of AVI_BSI please wait')
# Means Centered for VD --- VD WILL GIVE YOU ERROR ---> if you dont masked out first
Means_cn = means_centered(AVI_BSI, AOI, pca_scale, tileScale)
#print('Means_cn[0]',Means_cn[0])
#print('Means_cn[1]',Means_cn[1])

print('Now we proceed to the PCA of Vegetation density')
# PCA FOR AVI AND BSI = VD and SVI
VD = getPrincipalComponents(Means_cn[0], pca_scale, AOI, Means_cn[1]).rename(['VD1', 'VD2'])
#Map.addLayer(VD,{},'VD')
# scale VD datasets
SVI = normalization_100(VD,pca_scale=pca_scale, AOI=AOI)
Map.addLayer(SVI,{'min':0,'max':100},'SVI', False)
print('Success get the PCA normalized of VD => SVI')

print(f'no thermal band, choosing {I_satellite} images')
si = si_norm
#minsi =si.reduceRegion(reducer=ee.Reducer.min(),geometry=AOI,scale=1000,bestEffort=True,tileScale=16)
#minsi = ee.Dictionary(minsi).toImage()
#maxsi =si.reduceRegion(reducer=ee.Reducer.max(),geometry=AOI,scale=1000,bestEffort=True,tileScale=16)
#maxsi = ee.Dictionary(maxsi).toImage()

#ssiscale = si.subtract(minsi).divide(maxsi.subtract(minsi)).multiply(100)
#print(ssiscale, 'SSI')
Map.addLayer(si,{'min':0, 'max':100}, 'SSI', False)

print('Now calculating the FCD from SVI and SSI - selecting band svi1 svi2 ssi1 and ssi2')
svi1 = SVI.select(['VD1'])
ssi1 = si.select(['SI'])
svi2 = SVI.select(['VD2'])

# Starting to include in the formula for Forest Cover Density
FCD1_1 = (((svi1.multiply(ssi1)).add(1)).pow(0.5)).subtract(1).rename('FCD')
Map.addLayer(FCD1_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_1')
#Map.addLayer(FCD1.mask(FCD1.gte(30)), {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_morethan30')


print('Now calculating the FCD from SVI2 and SSI1 into FCD in percentage')
# The best on AOI_B30 based on the visual check, combination svi2 and ssi1
FCD2_1 = (((svi2.multiply(ssi1)).add(1)).pow(0.5)).subtract(1).rename('FCD')

Map.addLayer(FCD2_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_1')
#Map.addLayer(FCD2_1.mask(FCD2_1.gte(30)), {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_1_morethan30')
print('finish processing PCA please continue')

processing AVI
processing BSI
processing SI
Normalizing to 100 AVI
Normalizing to 100 BSI
Normalizing to 100 SI
Combining AVI AND BSI
Processing means center of AVI_BSI please wait
Now we proceed to the PCA of Vegetation density
Success get the PCA normalized of VD => SVI
no thermal band, choosing Landsat images
Now calculating the FCD from SVI and SSI - selecting band svi1 svi2 ssi1 and ssi2
Now calculating the FCD from SVI2 and SSI1 into FCD in percentage
finish processing PCA please continue


In [250]:
############################ Deforestation areas - TCL (Tree Cover Loss)
### adapted from https://developers.google.com/earth-engine/tutorials/community/forest-cover-loss-estimation
# hansen - updated version - global data
gfc = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

#Canopy cover percentage (e.g. 30%), for Indonesia
cc = ee.Number(tree_cover_forest)

#Minimum forest area in pixels (e.g. 3 pixels, ~ 0.27 ha in this example).
pixels = ee.Number(pixel_number)

#Minimum mapping area for tree loss (usually same as the minimum forest area).
lossPixels = pixels

canopyCover = gfc.select(['treecover2000'])
canopyCoverThreshold = canopyCover.gte(cc).selfMask()

#Use connectedPixelCount() to get contiguous area.
contArea = canopyCoverThreshold.connectedPixelCount()
#Apply the minimum area requirement.
minArea = contArea.gte(pixels).selfMask()

prj = gfc.projection()
scale = prj.nominalScale()
ForestArea2000Hansen = minArea.reproject(prj.atScale(scale))

#Map.addLayerControl()
Map.addLayer(ForestArea2000Hansen, {
    'palette': ['#96ED89']
}, 'tree cover: >= min canopy cover & area (light green)', False)

#create visual boundary color only
empty = ee.Image().byte()
AOIm = empty.paint(AOI,0,10)

##### Visualized LANDSAT CLOUDLESS FROM HANSEN DATASET LAST YEAR # just to check the visual images from Hansen
LastImageLandsat = gfc.select(['last_b30', 'last_b40', 'last_b50', 'last_b70']).rename(['red', 'nir', 'swir1', 'swir2']).clip(AOI)
Map.addLayer(LastImageLandsat, {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 600, 'gamma': 1.5 },
                f'Landsat Hansen Last Year {end_date}', False)

### HISTORICAL DATA THRESHOLD - Get the TCL data LOSS PIXEL overall data from loss year in the input
# adapted from https://developers.google.com/earth-engine/tutorials/community/forest-cover-loss-estimation

treeLossYear = gfc.select(['lossyear'])
treeLoss = treeLossYear.gte(year_start_loss).selfMask() # tree loss in e.g., year > 2012 ####SHOULD CHANGE TO RECENT YEAR for the '12' number
#Select the tree loss within the derived tree cover
#(>= canopy cover and area requirements). THIS ONE ALREADY MASK TCL IN FOREST AREA
treecoverLoss = minArea.And(treeLoss).rename(f'lossfrom_{year_start_loss}').selfMask()

#Create connectedPixelCount() to get contiguous area.
contLoss = treecoverLoss.connectedPixelCount()
#Apply the minimum area requirement, and get the TCL data ---> minLoss - ACTUAL TCL AREA from Hansen since the year_start_loss
minLoss = contLoss.gte(lossPixels).selfMask()

Map.addLayer(treeLossYear.clip(AOI),{"opacity":1,"bands":["lossyear"],"min":1,"max":22,"palette":["3d358c","4457c9","4777f0","4196ff","2eb4f3","1ad1d5","1ae5b6","36f493","64fd6a","92ff47","b4f836","d3e935","ecd239","fbb938","fe992c","f9751d","ec520e","d93806","bf2102","9f1001","7a0403","#FF0000"]},"Tree Loss Year", False)
Map.addLayer(minLoss.clip(AOI),{
    'palette': ['#ff0000']
}, f'tree cover >30% loss since 20{year_start_loss}', False)


In [251]:
## CHOOSE WHICH FCD map prefered
FCD = ee.Image(FCD2_1.select(['FCD'])) # here we use comb FCD2_1 = SVI 2, and SI 1 if
Map.addLayer(FCD,{'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']},'FCD_SELECTED')

print('Success calculating FCD best combination, and added to the map')

print('Now we label the FCD')
# Starting to create threshold for labeling
WaterMask = gfc.select(['datamask']).rename('datamask').eq(2)
WaterinAOI = AOI_img.mask().updateMask(WaterMask)
# unmasked the water (hansen)
unmaskedWaterAOI = AOI_img.unmask().updateMask(WaterinAOI.mask().Not()).clip(AOI)
#Map.addLayer(unmaskedWaterAOI,{},'unmasked_water_test', False)

######### Zone 5: Water Body - NDWI Sentinel
# from a function, output is already assigned in a band name: Class : 5
#water_list = applying_sentinel_water(image_corrected, ndwi_hi)
#waterinAOI_ndwi = water_list[0]
#unmaskedWaterAOI_ndwi = AOI_img.unmask().updateMask(waterinAOI_ndwi.mask().Not()).clip(AOI)


print(f'Adding the map of Forest, FCD >= {yrf_forest}% and mask only if not water in area (hansen) and NDWI')
#Forest
AllForest = FCD.mask(FCD.gte(yrf_forest)).And(unmaskedWaterAOI)
HighForestDense = FCD.mask(FCD.gte(high_forest)).And(unmaskedWaterAOI)
YRFForestDense = FCD.mask(FCD.gte(yrf_forest).And(FCD.lt(high_forest))).And(unmaskedWaterAOI)
#Map.addLayer(AllForest,{'min':0 ,'max':100}, 'AllForest')
Map.addLayer(HighForestDense, {'min':0 ,'max':100}, f'Forest (FCD >= {high_forest}%)', False)

Map.addLayer(YRFForestDense, {'min':0 ,'max':100}, f'Forest (FCD >= {yrf_forest}%) and < {high_forest}%', False)

print(f'Adding the map of Shrubland, FCD <  {yrf_forest}% and FCD >= {shrub_grass}%')
#Shrubland
Shrubland = FCD.mask(FCD.lt(yrf_forest).And(FCD.gte(shrub_grass))).And(unmaskedWaterAOI)
Map.addLayer(Shrubland, {'min':0 ,'max':100}, f'Shrubland ({shrub_grass}% <= FCD < {yrf_forest}%)', False)

print(f'Adding the map of Grassland or Openland, FCD  < {shrub_grass}%')
#Grassland / openland
Grassland = FCD.mask(FCD.lt(shrub_grass)).And(unmaskedWaterAOI)
Map.addLayer(Grassland, {'min':0 ,'max':100}, f'Grassland (FCD < {shrub_grass})', False)

hiforest_masked = AllForest
#Shrubland
#Grassland

print('Processing - the zoning classification')
highForestOnly = ee.Image(assigning_band(band_name_image,1,hiforest_masked.clip(AOI)))
#Map.addLayer(highForestOnly,{'bands': ['Class'],'palette':hi_forest},f'ALL_Forest_FCD > {yrf_forest}% - Class 1')

# Colors fo raster symbology
water = '#3380cc' #Blue
hi_forest = '#006837' #Strong green
low_forest = '#3ea540' #Medium green
grass_land = '#baf096' #Light green
bare_land = '#ad8855' #Brown
other = '#000000'

def applying_zone_sentinel_hansen(hiforest_masked,minLoss,s2_sr_median, ndwi_hi):

    ######### Zone 2 - High Baseline Forest: That does not have (overlaid with) TCL
    # Unmasked the forest loss 10 years rule
    unmaskedLoss = AOI_img.unmask().updateMask(minLoss.mask().Not()).clip(AOI)
    highBaselineF = hiforest_masked.And(unmaskedLoss)
    #Map.addLayer(highBaselineF,{'palette':hi_forest},'High baseline (Forest)')

    # assigning class band 2
    highf_edited = ee.Image(assigning_band(band_name_image,2,highBaselineF)) #high baseline forest - 2
    ##################################################

    ######### Zone 3: High Forest and Loss (e.g., Young Regenerating Forest)
    ####### Get the overlay information of HighBaseline (Sentinel) and Tree Cover Loss (Hansen), e.g., Young Regenerating Forest
    HiForestAndLoss = AOI_img.And(hiforest_masked.And(minLoss)) #minLoss is the actual TCL without overlaying with Sentinel
    #Map.addLayer(HiForestAndLoss,{'palette': '#FF0000'},"High Baseline and not pass 10 years rule")

    # Final Result for High Forest and TCL -> Class number: 3
    tenyrfl_edited = ee.Image(assigning_band(band_name_image,3,HiForestAndLoss)) #tenyears rule not pass and high baseline - 3
    ##############################################

    ######### Zone 4: Tree Cover Loss on the current low baseline (Sentinel)
    ###### Get the 10 years data only that is not overlay with Sentinel High baseline (Forest) #############
    # Create a helper mask indicating where the smaller areas maskhiFL, distinguish only the highbaseline and TCL (mask) and assign mask as 1
    maskHiFL = HiForestAndLoss.mask()
    #Map.addLayer(maskHiFL)
    # Helper - Invert the mask to get the areas where the smaller raster is absent - get only area outside high forest and loss, and mask from 1 to 0
    # invert mask just to get the number from 0 to 1 or vice versa -> .Not()
    maskHiFL_inverted = maskHiFL.Not()
    #Map.addLayer(maskHiFL_inverted)
    # Unmask the bigger raster in the areas to get the pixel value for area 'outside' HiFL (High Baseline and Forest Loss)
    unmaskedHiFL = AOI_img.unmask().updateMask(maskHiFL_inverted).clip(AOI)
    #Map.addLayer(unmaskedHiFL)
    #outcome result for 10 years data only that is not overlay with Sentinel High baseline (Forest) unmaskedHiFL is Area that is not Forest Sentinel############
    tenYearsRule = unmaskedHiFL.And(minLoss)
    #Map.addLayer(tenYearsRule,{'palette': '#FFA500'},"10 Years Rule - not OK")

    # assign the Class into no. 4 - Final Result
    tenyrule_edited = ee.Image(assigning_band(band_name_image,4,tenYearsRule)) #tenyears rule not pass - 4
    ###############################################

    ######### Zone 5: Water Body - NDWI Sentinel
    # from a function, output is already assigned in a band name: Class : 5
    #water_list = applying_sentinel_water(s2_sr_median, ndwi_hi)
    #waterinAOI = water_list[0]
    #water_edited = water_list[1]

    # Use hansen data instead - better result, no need to adjust threshold and water body not change for long time
    WaterMask = gfc.select(['datamask']).rename('datamask').eq(2)
    waterinAOI = AOI_img.mask().updateMask(WaterMask)
    water_edited = ee.Image(assigning_band(band_name_image,5,waterinAOI)).clip(AOI)
    # unmasked the water (hansen)
    unmaskedWaterAOI = AOI_img.unmask().updateMask(waterinAOI.mask().Not()).clip(AOI)

    ###############################################


    ######### Zone 1: Go Zone - outside (forest, forest - TCL, no-forest TCL, WaterBody)
    # Utils - Similar with above (but easier), unmasked the high forest means that in the end only include area that is not in forest (hiforest_masked)
    maskHiF = hiforest_masked.mask()
    maskHiF_inverted = maskHiF.Not()
    # Unmasked High Forest - the result is all the area outside of hiforest_masked (Total all forest), and now in no forest
    unmaskedHiF = AOI_img.unmask().updateMask(maskHiF_inverted).clip(AOI)
    # unmasked the water
    #unmaskedWaterAOI = AOI_img.unmask().updateMask(waterinAOI.mask().Not()).clip(AOI)

    goZone = unmaskedLoss.And(unmaskedHiF).And(unmaskedWaterAOI)
    #Map.addLayer(goZone,{'palette':'#FFFF00'},'Go Zone')
    # assigning band - Go Zone
    goZone_edited = ee.Image(assigning_band(band_name_image,1,goZone)) #go Zone - 1
    ##############################################

    Map.addLayer(goZone_edited,{'bands': ['Class'],'palette':'#FFFF00'},'Go Zone', False)
    Map.addLayer(highf_edited,{'bands': ['Class'],'palette':hi_forest},'High baseline (Forest)', False)
    Map.addLayer(tenyrfl_edited,{'bands': ['Class'],'palette': '#FF0000'},"High Baseline and not pass 10 years rule", False)
    Map.addLayer(tenyrule_edited,{'bands': ['Class'],'palette': '#FFA500'},"10 Years Rule - not OK", False)
    Map.addLayer(water_edited,{'bands': ['Class'],'palette': water},"Water in AOI", False)


    ''' mosaicking into single image  - still does not work
    #image_list = ee.List([goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited ])
    # Extract the spatial information from one of the individual images.
    projection = goZone_edited.projection()
    scale = goZone_edited.projection().nominalScale()

    # Create an ee.ImageCollection from the list of images
    image_collection = ee.ImageCollection([goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited ])

    # Merge the images into a single ee.Image
    zoning_final = image_collection.mosaic()
    '''
    return [goZone_edited,highf_edited,tenyrfl_edited,tenyrule_edited, water_edited]

tiff_result = applying_zone_sentinel_hansen(hiforest_masked,minLoss,image_mosaick, ndwi_hi)

# HighForestDense = FCD.mask(FCD.gte(high_forest).And(unmaskedWaterAOI))
HighForestDense_no10yrs = HighForestDense.And(tiff_result[1]).select(['pixel'])
HighForestDense_no10yrs = ee.Image(assigning_band(band_name_image,21,HighForestDense_no10yrs))

HighForestDense_10yrs = HighForestDense.And(tiff_result[2]).select(['pixel'])
HighForestDense_10yrs = ee.Image(assigning_band(band_name_image,31,HighForestDense_10yrs))

#YRFForestDense = FCD.mask(FCD.gte(yrf_forest).And(unmaskedWaterAOI).And(FCD.lt(high_forest)))
YRFForestDense_no10yrs = YRFForestDense.And(tiff_result[1]).select(['pixel'])
YRFForestDense_no10yrs = ee.Image(assigning_band(band_name_image,22,YRFForestDense_no10yrs))

YRFForestDense_10yrs = YRFForestDense.And(tiff_result[2]).select(['pixel'])
YRFForestDense_10yrs = ee.Image(assigning_band(band_name_image,32,YRFForestDense_10yrs))

Shrubland_gozone = Shrubland.And(tiff_result[0]).select(['pixel'])
Shrubland_gozone = ee.Image(assigning_band(band_name_image,11,Shrubland_gozone))

Grassland_gozone = Grassland.And(tiff_result[0]).select(['pixel'])
Grassland_gozone = ee.Image(assigning_band(band_name_image,12,Grassland_gozone))

# Filter the images based on the 'Class' band
grassland_gozone = Grassland_gozone.select(['Class'])
Map.addLayer(grassland_gozone,{'palette': ['#ffff33']},'Grass land - Go Zone')

shrubland_gozone = Shrubland_gozone.select(['Class'])
Map.addLayer(shrubland_gozone,{'palette': ['#ffe3b3']},'Shrub land - Go Zone')

yrf_forest_dense_no10yrs = YRFForestDense_no10yrs.select(['Class'])
Map.addLayer(yrf_forest_dense_no10yrs,{'palette': ['#83ff5a']},'Low-Med Forest')

high_forest_dense_no10yrs = HighForestDense_no10yrs.select('Class')
Map.addLayer(high_forest_dense_no10yrs,{'palette': ['#09ab0c']},'High- Density Forest')

yrf_forest_dense_10yrs = YRFForestDense_10yrs.select(['Class'])
Map.addLayer(yrf_forest_dense_10yrs,{'palette': ['#ff0abe']},'Re-growth Low-Med Density Forest')

high_forest_dense_10yrs = HighForestDense_10yrs.select(['Class'])
Map.addLayer(high_forest_dense_10yrs,{'palette': ['#ff0004']},'Re-growth High Density Forest')

No_pass_historical_years_rule = tiff_result[3].select(['Class'])
Map.addLayer(No_pass_historical_years_rule,{'palette': ['#ff8a1d']},'Historical deforestation no regrowth (3 or 10 years)')

Water_Un_plantable = tiff_result[4].select(['Class'])
Map.addLayer(Water_Un_plantable,{'palette': ['#1900ff']},'Water body (unplantable)')


empty_image = ee.Image.constant(0).rename('Class')

def add_classes(image, empty):
    # Cast the image to Int32 to handle NaN and null values
    image = image.unmask(0).toInt32()
    # Set a conditional statement to retain existing class values if non-zero, otherwise use the new class value
    merged_image = empty.where(empty.neq(0), empty).where(image.neq(0), image)
    return merged_image

# Create a list of images excluding the 'empty_image'
image_list = [
    grassland_gozone,
    shrubland_gozone,
    yrf_forest_dense_no10yrs,
    high_forest_dense_no10yrs,
    yrf_forest_dense_10yrs,
    high_forest_dense_10yrs,
    No_pass_historical_years_rule,
    Water_Un_plantable
]

# Create an ImageCollection from the list of images
image_collection = ee.ImageCollection(image_list)

# Apply the add_classes function to each image in the collection while merging them into the 'empty_image'
result_collection = image_collection.map(lambda image: add_classes(image, empty_image))

# Create a function to add two images element-wise
def add_images(img1, img2):
    return ee.Image(img1).add(img2)

# Merge all the images in the result_collection using ee.ImageCollection.iterate()
merged_image = ee.Image(result_collection.iterate(add_images, empty_image))

# Cast the merged image to Int32 and set the original Class band name
merged_image = merged_image.toInt32().rename('Class')
merged_image = merged_image.clip(AOI)
print('finish processing')


Success calculating FCD best combination, and added to the map
Now we label the FCD
Adding the map of Forest, FCD >= 45% and mask only if not water in area (hansen) and NDWI
Adding the map of Shrubland, FCD <  45% and FCD >= 25%
Adding the map of Grassland or Openland, FCD  < 25%
Processing - the zoning classification
finish processing


In [253]:
# Load DEM data (replace 'dataset' with your actual DEM dataset)
DEM = ee.Image('USGS/SRTMGL1_003').clip(AOI)

# Calculate slope in degrees
slope = ee.Terrain.slope(DEM)

# Convert slope to percentage
slopePercentage = slope.expression('tan(b*0.01745) * 100', {'b': slope})

# Define slope classification thresholds
thresholds = [8, 15, 25, 40]  # Adjust these thresholds as needed

# Classify slope into categories using conditional statements
slopeClasses = slopePercentage \
    .lte(thresholds[0]).multiply(1) \
    .add(slopePercentage.gt(thresholds[0]).And(slopePercentage.lte(thresholds[1])).multiply(2)) \
    .add(slopePercentage.gt(thresholds[1]).And(slopePercentage.lte(thresholds[2])).multiply(3)) \
    .add(slopePercentage.gt(thresholds[2]).And(slopePercentage.lte(thresholds[3])).multiply(4)) \
    .add(slopePercentage.gt(thresholds[3]).multiply(5))

# Display the classified slope image
palette = ['lightgreen', 'yellow', 'orange', 'pink', 'red']  # Change green to lightgreen
vis_params = {'min': 1, 'max': 5, 'palette': palette}
Map.addLayer(slopeClasses, vis_params, 'Slope Classes', True)
# Map.addLayer(DEM)
#Map.addLayer(Grassland_gozone.And(slopeClasses.eq(1).selfMask()),{'min':1,'max':100}, 'Grassland Slope 1')
#Map.addLayer(Grassland_gozone.And(slopeClasses.eq(2).selfMask()),{'min':1,'max':100}, 'Grassland Slope 2')
#Map.addLayer(Grassland_gozone.And(slopeClasses.eq(3).selfMask()),{'min':1,'max':100},'Grassland Slope 3')
#Map.addLayer(Grassland_gozone.And(slopeClasses.eq(4).selfMask()),{'min':1,'max':100},'Grassland Slope 4')
#Map.addLayer(Grassland_gozone.And(slopeClasses.eq(5).selfMask()),{'min':1,'max':100},'Grassland Slope 5')

class_name_map = {
    '11': 'Shrubland_Go-Zone',
    '12': 'Grassland_Go-Zone',
    '21': 'High density Forest',
    '22': 'Low - Medium density Forest',
    '31': 'Regrowth High Density Forest from deforested (historical)',
    '32': 'Regrowth Low Density Forest from deforested (historical)',
    '4': 'Historical deforestation (3 or 10 years rule)',
    '5': 'Water Body'
}

key_value_strings = [f'{key}: {value}' for key, value in class_name_map.items()]

class_name_map_color = {
    '11': '#ffe3b3',
    '12': '#ffff33',
    '21': '#09ab0c',
    '22': '#83ff5a',
    '31': '#ff0004',
    '32': '#ff0abe',
    '4': '#ff8a1d',
    '5': '#1900ff'
}

value_list = [value for key, value in class_name_map_color.items()]

labels = key_value_strings
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
colors = value_list
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

Map.add_legend(title='Legend', labels=labels, colors=colors, position='bottomleft')
Map.addLayerControl()

In [254]:
# Exporting to tiff
##### INPUT ###########################
title_map = 'Axis Landscape'

AOI_NAME = title_map
START_DATE = start_date
END_DATE = end_date

list_images = [FCD,Grassland_gozone,Shrubland_gozone , YRFForestDense_no10yrs,
                HighForestDense_no10yrs, YRFForestDense_10yrs, HighForestDense_10yrs,
              tiff_result[3], tiff_result[4], slopeClasses]
list_names =  ['FCD','Go Zone_Grassland', 'Go Zone_Shrubland', 'YRF_Forest',
                'High_Dense_Forest','YRF_nopass_historical', 'HDenseForest_nopass_historical',
                'No_pass_historical_years_rule', 'Water_Un_plantable','Slope-Classes']
for i in range(len(list_images)):
    geemap.ee_export_image_to_drive(
        list_images[i], description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_{list_names[i]}', folder=f"JupyterGEE_{AOI_NAME}", scale=pca_scale, region=AOI.geometry(),
    )
    print(f'exported - {list_names[i]}')

# if I_satellite == 'LANDSAT':
#     geemap.ee_export_image_to_drive(
#             image_corrected.clip(AOI), description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_LANDSAT', folder=f"JupyterGEE_{AOI_NAME}", scale=pca_scale, region=AOI.geometry(),
#         )
#     print(f'exported - LANDSAT')

# elif TI_include == False and satellite == 'planet':
#     geemap.ee_export_image_to_drive(
#             filtered_scaled.clip(AOI), description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_PlanetNICFI_MEDIAN', folder=f"JupyterGEE_{AOI_NAME}", scale=pca_scale, region=AOI.geometry(),
#         )
#     print(f'exported - Planet Labs')

# elif TI_include == False and satellite == 'sentinel':
#     geemap.ee_export_image_to_drive(
#             image_corrected.clip(AOI), description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_Sentinel_median', folder=f"JupyterGEE_{AOI_NAME}", scale=pca_scale, region=AOI.geometry(),
#         )
#     print(f'exported - Sentinel')

# Print the resulting merged image
#print(merged_image.getInfo())


geemap.ee_export_image_to_drive(
            merged_image.select('Class').toInt(), description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_merged', folder=f"JupyterGEE_{AOI_NAME}", scale=pca_scale, region=AOI.geometry(),
        )
print(f'exported - Merged_classes')

exported - FCD
exported - Go Zone_Grassland
exported - Go Zone_Shrubland
exported - YRF_Forest
exported - High_Dense_Forest
exported - YRF_nopass_historical
exported - HDenseForest_nopass_historical
exported - No_pass_historical_years_rule
exported - Water_Un_plantable
exported - Slope-Classes
exported - Merged_classes


In [207]:
print(type(conf_selected.start_date_analysis))

<class 'datetime.date'>


In [166]:
from asgiref.sync import sync_to_async

async def async_filter():
    try:
        label = await sync_to_async(SatVerConfiguration.objects.get)(label="TEST_DEMO_TREEO_Landsat_2022")
    except SatVerConfiguration.DoesNotExist:
        print('label object is not found')
        label = None
    return label

query = await async_filter()

In [169]:
query.id

58